* 目的
    * 爬取<font color="red">淘寶</font>手機熱銷榜
* 分析
    * 淘寶網站為動態網頁，使用selenium驅動headless的Chrome執行頁面中的javascript，模擬真實瀏覽的過程


In [127]:
import pandas as pd
import json
import requests
import time
import gc
import sys

In [128]:
# selenium教學
# http://uule.iteye.com/blog/2370372
from bs4 import BeautifulSoup
from selenium import webdriver
DEBUG = False

In [129]:
option = webdriver.ChromeOptions()
if not DEBUG:
    option.set_headless()
    option.add_argument('headless')
option.add_argument("--incognito")
browser = webdriver.Chrome(
    executable_path='../../chromedriver.exe',
    chrome_options=option
)

In [130]:
# 1st stage query: 取得項目id listz
type_text = '手机'
display_by = 'type_text'
url = 'https://s.taobao.com/search?q={}&psort={}'.format(type_text, display_by)

頁面隨著畫面捲動讀取，所以需要模擬手動捲動的過程，觸發頁面的腳本把沒載入的地方載入

In [131]:
def scroll_page():
    height = browser.execute_script('''return document.body.scrollHeight''')
    intv = 1000
    for i in range(0, height, intv):
        browser.execute_script('''
            window.scrollTo(0, {});
            return undefined;
        '''.format(i))
        time.sleep(0.5)

In [132]:
browser.get(url)
browser.maximize_window()
scroll_page()

In [133]:
READ_PAGES = 5

個人比較習慣使用javascript爬資料
selenium支援在受控瀏覽器執行js

### 擷取頁面的資料

In [134]:
JS_GET_ITEMS_INFO = '''
function get() {
    var output = [];
    // 取得商品項目的DOM節點
    var items = document.getElementsByClassName('grid-panel');
    for(var i = 0; i < items.length; ++i)
    {
        try {
            var temp0 = items[i];
            var obj = new Object();
            obj['screen'] = temp0.children[0].children[0].innerText;
            var title = temp0.children[1].children[0];
            obj['title'] = title.children[0].childNodes[0].data.trimLeft().trimRight();
            obj['features'] = [];
            var features = title.children[0].childNodes[1].children;
            for(var j = 0; j < features.length; ++j)
            {
                var item = features[j];
                obj['features'].push(item.innerText);
            }
            obj['price'] = temp0.children[1].children[1].children[0].children[1].children[1].innerText;
            obj['bought'] = temp0.children[1].children[1].children[1].children[0].children[0].innerText;
            obj['seller'] = temp0.children[1].children[3].children[0].innerText;
            output.push(obj);
        }
        catch{};
    }
    // 轉換成字串返回
    return JSON.stringify(output);
}
return get();
'''

In [135]:
browser.execute_script('''
    window.scrollTo(0,document.body.scrollHeight);
''')
time.sleep(5)
json.loads(browser.execute_script(JS_GET_ITEMS_INFO)).__len__()

48

一次爬多頁

In [136]:
items = []
for i in range(READ_PAGES):
    items += json.loads(browser.execute_script(JS_GET_ITEMS_INFO))
    browser.execute_script('''
        // Switch page
        var pager = document.getElementsByClassName('J_Ajax num icon-tag');
        pager[pager.length - 1].click();
    ''')
    scroll_page()


要記得手動關閉瀏覽器，不然就會一直留在記憶體裡面占用資源

In [137]:
browser.close()
browser.quit()

In [138]:
df = pd.DataFrame(items)

In [139]:
df['bought'] = df['bought'].astype(int, copy=False)

In [140]:
df.sort_values(by='bought', axis=0, inplace=True, ascending=False)
df.reset_index(inplace=True)
df

,index,bought,features,price,screen,seller,title
0,24,200599,"[双镜头光学变焦, 原彩显示技术, 全新感光元件]",4858,"大屏5.5""",共有963商家在售,苹果 iPhone 8 Plus
1,22,123507,"[面容ID识别, 前置深感摄像头, 人像光效]",6685,"大屏5.8""",共有890商家在售,苹果 iPhone X
2,3,107785,"[前后2000万，自拍更美, 骁龙660 AIE处理器]",1410,"大屏5.99""",共有306商家在售,小米 6X
3,9,98845,"[3D炫彩流光机身, 后置1300万AI智慧双摄]",1596,"超大屏6.26""",共有57商家在售,vivo Z1
4,6,97693,"[2400万AI摄影, 3D人像光效, 超级快充]",2525,"大屏5.84""",共有373商家在售,荣耀 荣耀10
5,29,93644,"[AI双摄, 骁龙636高性能处理器, 暗光逆光更出色]",873,"大屏5.99""",共有347商家在售,小米 红米Note 5
6,23,84084,"[双面全玻璃设计, 抗水防尘, 支持无线充电]",4498,"中屏4.7""",共有726商家在售,苹果 iPhone 8
7,202,83271,"[强固屏幕, 充电宝功能]",107,"小屏2.4""",共有9商家在售,尼凯恩 EN3
8,103,80985,"[两个镜头，一拍，即合, 防水防尘]",3237,"大屏5.5""",共有1390商家在售,苹果 iPhone 7 Plus
9,153,77458,"[光学防抖, 3DTouch触摸]",2706,"大屏5.5""",共有1033商家在售,苹果 iPhone 6s Plus
